# Listwise Reranking using OpenAI

![Listwise ranking](./listwise.jpeg)

#### Listwise reranking takes all the retrieved documents and query as input to the ranker. Listwise reranking uses prompt engineering to feed in the input of the retrieved documents + query and returns a structured output of the results [Doc B > Doc C > Doc A]. The objective of the LLM is to find the best document ordering that maximizes the retrieval metric (i.e. nDCG, precision).

### Connect to Weaviate instance

In [1]:
import weaviate
import os

client = weaviate.connect_to_embedded()

{"action":"startup","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-12-27T12:19:05-03:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-12-27T12:19:05-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-12-27T12:19:05-03:00"}
{"build_git_commit":"ab0312d5d","build_go_versio

In [2]:
print("Client Version:", weaviate.__version__, "Server Version:", client.get_meta().get("version"))

Client Version: 4.10.2 Server Version: 1.26.6


### Load in FAQ json

In [3]:
import json

with open("faq.json", "r") as f:
    json_data = json.load(f)

queries = [{"question": item["question"], "answer": item["answer"], "number": item["number"]} for item in json_data["questions"]]

### Create Weaviate Schema

In [4]:
from weaviate import classes as wvc

# resetting the schema. CAUTION: THIS WILL DELETE YOUR DATA 
client.collections.delete("FAQ_Answers")

collection = client.collections.create(
    name="FAQ_Answers",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    properties=[
        wvc.config.Property(name="Answer", data_type=wvc.config.DataType.TEXT),
        wvc.config.Property(name="Number", data_type=wvc.config.DataType.TEXT),
    ]
)
print("Successfully created the schema.")

Successfully created the schema.


#### Upload answers to Weaviate

In [5]:
from weaviate.util import get_valid_uuid
from uuid import uuid4

for item in queries:
  print(item)
  properties = {
    "Answer": item["answer"],
    "Number": item["number"]
  }
  id = get_valid_uuid(uuid4())
  collection.data.insert(properties, uuid=id)

{'question': 'Why would I use Weaviate as my vector database?', 'answer': 'Our goal is three-folded. Firstly, we want to make it as easy as possible for others to create their own semantic systems or vector search engines (hence, our APIs are GraphQL based). Secondly, we have a strong focus on the semantic element (the knowledge in vector databases, if you will). Our ultimate goal is to have Weaviate help you manage, index, and understand your data so that you can build newer, better, and faster applications. And thirdly, we want you to be able to run it everywhere. This is the reason why Weaviate comes containerized.', 'number': '1'}
{'question': 'What is the difference between Weaviate and for example Elasticsearch?', 'answer': 'Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is store

### Retrieve information

##### Prompt 1

In [6]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

def openai_request(prompt):
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
        {
        "role": "system",
        "content": """
        You are a reranking agent. Each potential answer has a corresponding Answer id and you're tasked with ranking the questions based on their relevancy to the query.
        The output should ONLY contain the ranked list and no additional comments.
        """
        },
        {
            "role": "user",
            "content": prompt
        }],
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [ ]:
queries

In [7]:
def parse_ranked_ids(ids):
    elements = ids.split(',')

    first_element = elements[0].strip('[]')

    return first_element


correct = 0
it = 0
for query_obj in queries:
    it += 1
    print("ITERATIONS", it)
    query = query_obj["question"]
    ground_truth = int(query_obj["number"])

    results = collection.query.near_text(
        query=query,
        limit=10
    )
    print(f"Weaviate search results {len(results.objects)}\n")

    reranking_template = f"\nINPUT:\nQUERY: {query}"
    reranking_template += "\nPlease rerank these search results.\n"
    for result in results.objects:
        id, answer = result.properties["number"], result.properties["answer"]
        reranking_template += f"[Answer id: {id}, Answer: {answer}]\n"
    print(reranking_template)
    print("\n")
    ranked_ids = openai_request(reranking_template) # send the query along with retrieved results to OpenAI
    first_ranking = parse_ranked_ids(ranked_ids)
    print(f"OPENAI 1st Rank = {first_ranking}\n") # first ranking from gpt-4
    print(f"GROUND TRUTH = {ground_truth}") # ground truth
    if (first_ranking == ground_truth):
        correct += 1

print(correct / len(queries) * 100)


ITERATIONS 1
Weaviate search results 10


INPUT:
QUERY: Why would I use Weaviate as my vector database?
Please rerank these search results.
[Answer id: 2, Answer: Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is stored as vectors, which enables semantic search. This combination of data storage is unique, and enables fast, filtered and semantic search from end-to-end.]
[Answer id: 1, Answer: Our goal is three-folded. Firstly, we want to make it as easy as possible for others to create their own semantic systems or vector search engines (hence, our APIs are GraphQL based). Secondly, we have a strong focus on the semantic element (the knowledge in vector databases, if you will). Our ultimate goal is to have Weaviate help you manage, index, and understand your data so that you can build n

##### Prompt 2

In [8]:
def openai_request(prompt):
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
        {
        "role": "system", #systems prompt
        "content": """
            You are a reranking agent! Each potential answer has a corresponding Answer id and you're tasked with ranking the answers based on their relevancy to the QUERY.
            You SHOULD ONLY contain the ranked list and no additional comments, such as [8],[7],[3],[1],[5]. THIS IS VERY IMPORTANT!
            Here is an example of the task you should perform:
            INPUT:
            QUERY: Why would I use Weaviate as my vector database?
            Please rerank these search results.
            [Answer id: 3, Answer: Weaviate uses Docker images as a means to distribute releases and uses Docker Compose to tie a module-rich runtime together. If you are new to those technologies, we recommend reading the Docker Introduction for Weaviate Users.]
            [Answer id: 2, Answer: Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is stored as vectors, which enables semantic search. This combination of data storage is unique, and enables fast, filtered and semantic search from end-to-end.]
            [Answer id: 11, Answer: We use a schema because it focusses on the representation of your data (in our case in the GraphQL API) but you can use a Weaviate schema to express an ontology. One of Weaviate's core features is that it semantically interprets your schema (and with that your ontology) so that you can search for concepts rather than formally defined entities.]
            [Answer id: 1, Answer: Our goal is three-folded. Firstly, we want to make it as easy as possible for others to create their own semantic systems or vector search engines (hence, our APIs are GraphQL based). Secondly, we have a strong focus on the semantic element (the knowledge in vector databases, if you will). Our ultimate goal is to have Weaviate help you manage, index, and understand your data so that you can build newer, better, and faster applications. And thirdly, we want you to be able to run it everywhere. This is the reason why Weaviate comes containerized.]
            [Answer id: 14, Answer: Yes, Weaviate supports cursor-based iteration as well as pagination through a result set. To iterate through all objects, you can use the after operator with both REST and GraphQL. For pagination through a result set, you can use the offset and limit operators for GraphQL API calls. Take a look at this page which describes how to use these operators, including tips on performance and limitations.]
            [Answer id: 10, Answer: Yes, you can! Weaviate support ontology, RDF-like definitions in its schema, and it runs out of the box. It is scalable, and the GraphQL API will allow you to query through your knowledge graph easily. But now you are here. We like to suggest you really try its semantic features. After all, you are creating a knowledge graph 😉.]
            [Answer id: 5, Answer: As a rule of thumb, the smaller the units, the more accurate the search will be. Two objects of e.g. a sentence would most likely contain more information in their vector embedding than a common vector (which is essentially just the mean of sentences). At the same time more objects leads to a higher import time and (since each vector also makes up some data) more space. (E.g. when using transformers, a single vector is 768xfloat32 = 3KB. This can easily make a difference if you have millions, etc.) of vectors. As a rule of thumb, the more vectors you have the more memory you're going to need. So, basically, it's a set of tradeoffs. Personally we've had great success with using paragraphs as individual units, as there's little benefit in going even more granular, but it's still much more precise than whole chapters, etc. You can use cross-references to link e.g. chapters to paragraphs. Note that resolving a cross-references takes a slight performance penalty. Essentially resolving A1->B1 is the same cost as looking up both A1 and B1 indvidually. This cost however, will probably only matter at really large scale.]
            [Answer id: 8, Answer: The UUID must be presented as a string matching the Canonical Textual representation. If you don't specify a UUID, Weaviate will generate a v4 i.e. a random UUID. If you generate them yourself you could either use random ones or deterministically determine them based on some fields that you have. For this you'll need to use v3 or v5.]
            [Answer id: 7, Answer: Yes. Each class itself acts like namespaces. Additionally, you can use the multi-tenancy feature to create isolated storage for each tenant. This is especially useful for use cases where one cluster might be used to store data for multiple customers or users.]
            [Answer id: 13, Answer: To obtain the cosine similarity from weaviate's certainty, you can do cosine_sim = 2*certainty - 1]
            

            OUTPUT:
            [1],[2] 
        """
        },
        {
        "role": "user", # users prompt
        "content": prompt
        }],
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [9]:
def parse_ranked_ids(ids):
    elements = ids.split(',')

    first_element = elements[0].strip('[]')

    return int(first_element)


correct = 0
for query_obj in queries:
    query = query_obj["question"]
    ground_truth = int(query_obj["number"])

    results = collection.query.near_text(
        query=query,
        limit=10
    )
    print(f"Weaviate search results {results}\n")

    reranking_template = f"\nINPUT: \nQUERY: {query}"
    reranking_template += "\nPlease rerank these search results.\n"
    for result in results.objects:
        id, answer = result.properties["number"], result.properties["answer"]
        reranking_template += f"[Answer id: {id}, Answer: {answer}]\n"
    print(reranking_template)
    print("\n")
    ranked_ids = openai_request(reranking_template) # send the query along with retrieved results to OpenAI
    print(f"RAW OUTPUT FROM OPENAI = {ranked_ids}\n")
    first_ranking = parse_ranked_ids(ranked_ids)
    print(f"OPENAI 1st Rank = {first_ranking}\n") # first ranking from gpt-4
    print(f"GROUND TRUTH = {ground_truth}") # ground truth
    if (first_ranking == ground_truth):
        correct += 1

print(correct / len(queries) * 100)


Weaviate search results QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('0ae59a26-a5d8-487d-803c-61e18d875a8d'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'number': '2', 'answer': 'Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is stored as vectors, which enables semantic search. This combination of data storage is unique, and enables fast, filtered and semantic search from end-to-end.'}, references=None, vector={}, collection='FAQ_Answers'), Object(uuid=_WeaviateUUIDInt('f2791143-c2ae-43d2-9917-1e57c0a25942'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_cons

##### Prompt 3

In [10]:
def openai_request(prompt):
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
        {
        "role": "system", #systems prompt
        "content": """
        You are a reranking agent. Each potential answer has a corresponding Answer id and you're tasked with ranking the questions based on their relevancy to the QUERY.
    VERY IMPORTANT!!! The output SHOULD ONLY contain the ranked list and no additional comments, such as [8],[7],[3],[1],[5]! THIS IS VERY IMPORTANT!
    Here is an example of the task you should perform:
    INPUT:
    QUERY: Why would I use Weaviate as my vector database?
    Please rerank these search results.
    [Answer id: 2, Answer: Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is stored as vectors, which enables semantic search. This combination of data storage is unique, and enables fast, filtered and semantic search from end-to-end.]
    [Answer id: 1, Answer: Our goal is three-folded. Firstly, we want to make it as easy as possible for others to create their own semantic systems or vector search engines (hence, our APIs are GraphQL based). Secondly, we have a strong focus on the semantic element (the knowledge in vector databases, if you will). Our ultimate goal is to have Weaviate help you manage, index, and understand your data so that you can build newer, better, and faster applications. And thirdly, we want you to be able to run it everywhere. This is the reason why Weaviate comes containerized.]
    [Answer id: 3, Answer: Weaviate uses Docker images as a means to distribute releases and uses Docker Compose to tie a module-rich runtime together. If you are new to those technologies, we recommend reading the Docker Introduction for Weaviate Users.]
    [Answer id: 11, Answer: We use a schema because it focusses on the representation of your data (in our case in the GraphQL API) but you can use a Weaviate schema to express an ontology. One of Weaviate's core features is that it semantically interprets your schema (and with that your ontology) so that you can search for concepts rather than formally defined entities.]
    [Answer id: 14, Answer: Yes, Weaviate supports cursor-based iteration as well as pagination through a result set. To iterate through all objects, you can use the after operator with both REST and GraphQL. For pagination through a result set, you can use the offset and limit operators for GraphQL API calls. Take a look at this page which describes how to use these operators, including tips on performance and limitations.]
    [Answer id: 10, Answer: Yes, you can! Weaviate support ontology, RDF-like definitions in its schema, and it runs out of the box. It is scalable, and the GraphQL API will allow you to query through your knowledge graph easily. But now you are here. We like to suggest you really try its semantic features. After all, you are creating a knowledge graph 😉.]
    [Answer id: 5, Answer: As a rule of thumb, the smaller the units, the more accurate the search will be. Two objects of e.g. a sentence would most likely contain more information in their vector embedding than a common vector (which is essentially just the mean of sentences). At the same time more objects leads to a higher import time and (since each vector also makes up some data) more space. (E.g. when using transformers, a single vector is 768xfloat32 = 3KB. This can easily make a difference if you have millions, etc.) of vectors. As a rule of thumb, the more vectors you have the more memory you're going to need. So, basically, it's a set of tradeoffs. Personally we've had great success with using paragraphs as individual units, as there's little benefit in going even more granular, but it's still much more precise than whole chapters, etc. You can use cross-references to link e.g. chapters to paragraphs. Note that resolving a cross-references takes a slight performance penalty. Essentially resolving A1->B1 is the same cost as looking up both A1 and B1 indvidually. This cost however, will probably only matter at really large scale.]
    [Answer id: 8, Answer: The UUID must be presented as a string matching the Canonical Textual representation. If you don't specify a UUID, Weaviate will generate a v4 i.e. a random UUID. If you generate them yourself you could either use random ones or deterministically determine them based on some fields that you have. For this you'll need to use v3 or v5.]
    [Answer id: 7, Answer: Yes. Each class itself acts like namespaces. Additionally, you can use the multi-tenancy feature to create isolated storage for each tenant. This is especially useful for use cases where one cluster might be used to store data for multiple customers or users.]
    [Answer id: 13, Answer: To obtain the cosine similarity from weaviate's certainty, you can do cosine_sim = 2*certainty - 1]

    OUTPUT:
    [1],[2] 
        """
        },
        {
            "role": "user", # users prompt
            "content": prompt
        }],
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [11]:
def parse_ranked_ids(ids):
    elements = ids.split(',')

    first_element = elements[0].strip('[]')

    return int(first_element)


correct = 0
for query_obj in queries:
    query = query_obj["question"]
    ground_truth = int(query_obj["number"])

    results = collection.query.near_text(
        query=query,
        limit=10
    )
    print(f"Weaviate search results {results}\n")
    

    reranking_template = f"\nINPUT: \nQUERY: {query}"
    reranking_template += "\nPlease rerank these search results.\n"
    for result in results.objects:
        id, answer = result.properties["number"], result.properties["answer"]
        reranking_template += f"[Answer id: {id}, Answer: {answer}]\n"
    print(reranking_template)
    print("\n")
    ranked_ids = openai_request(reranking_template) # send the query along with retrieved results to OpenAI
    print(f"RAW OUTPUT FROM OPENAI = {ranked_ids}\n")
    first_ranking = parse_ranked_ids(ranked_ids)
    print(f"OPENAI 1st Rank = {first_ranking}\n") # first ranking from gpt-4
    print(f"GROUND TRUTH = {ground_truth}") # ground truth
    if (first_ranking == ground_truth):
        correct += 1

print(correct / len(queries) * 100)

Weaviate search results QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('0ae59a26-a5d8-487d-803c-61e18d875a8d'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'number': '2', 'answer': 'Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is stored as vectors, which enables semantic search. This combination of data storage is unique, and enables fast, filtered and semantic search from end-to-end.'}, references=None, vector={}, collection='FAQ_Answers'), Object(uuid=_WeaviateUUIDInt('f2791143-c2ae-43d2-9917-1e57c0a25942'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_cons

##### Prompt 4

In [12]:
def openai_request(prompt):
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
        {
        "role": "system", #systems prompt
        "content": """
        You are a reranking agent. Each Answer has a corresponding Answer id and you're tasked with ranking the questions based on their relevancy to the QUERY.
        The output should ONLY contain the ranked list and no additional comments. An example of your output looks like this [8],[7],[3],[1],[5]!
        Here is an example of the task you will perform:
        ```
        INPUT:
        QUERY: Why would I use Weaviate as my vector database?
        Please rerank these search results.
        [Answer id: 3, Answer: Weaviate uses Docker images as a means to distribute releases and uses Docker Compose to tie a module-rich runtime together. If you are new to those technologies, we recommend reading the Docker Introduction for Weaviate Users.]
        [Answer id: 11, Answer: We use a schema because it focusses on the representation of your data (in our case in the GraphQL API) but you can use a Weaviate schema to express an ontology. One of Weaviate's core features is that it semantically interprets your schema (and with that your ontology) so that you can search for concepts rather than formally defined entities.]
        [Answer id: 14, Answer: Yes, Weaviate supports cursor-based iteration as well as pagination through a result set. To iterate through all objects, you can use the after operator with both REST and GraphQL. For pagination through a result set, you can use the offset and limit operators for GraphQL API calls. Take a look at this page which describes how to use these operators, including tips on performance and limitations.]

        OUTPUT:
        [14],[3] 
        ```
        Please consider all answers first before making your final decision!

        """
        },
        {
            "role": "user", # users prompt
            "content": prompt
        }],
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [13]:
def parse_ranked_ids(ids):
    elements = ids.split(',')

    first_element = elements[0].strip('[]')

    return int(first_element)


correct = 0
for query_obj in queries:
    query = query_obj["question"]
    ground_truth = int(query_obj["number"])

    results = collection.query.near_text(
        query=query,
        limit=10
    )
    print(f"Weaviate search results {results}\n")
    

    reranking_template = f"\nINPUT: \nQUERY: {query}"
    reranking_template += "\nPlease rerank these search results.\n"
    for result in results.objects:
        id, answer = result.properties["number"], result.properties["answer"]
        reranking_template += f"[Answer id: {id}, Answer: {answer}]\n"
    print(reranking_template)
    print("\n")
    ranked_ids = openai_request(reranking_template) # send the query along with retrieved results to OpenAI
    print(f"RAW OUTPUT FROM OPENAI = {ranked_ids}\n")
    first_ranking = parse_ranked_ids(ranked_ids)
    print(f"OPENAI 1st Rank = {first_ranking}\n") # first ranking from gpt-4
    print(f"GROUND TRUTH = {ground_truth}") # ground truth
    if (first_ranking == ground_truth):
        correct += 1

print(correct / len(queries) * 100)

Weaviate search results QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('0ae59a26-a5d8-487d-803c-61e18d875a8d'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'number': '2', 'answer': 'Other database systems like Elasticsearch rely on inverted indices, which makes search super fast. Weaviate also uses inverted indices to store data and values. But additionally, Weaviate is also a vector-native search database, which means that data is stored as vectors, which enables semantic search. This combination of data storage is unique, and enables fast, filtered and semantic search from end-to-end.'}, references=None, vector={}, collection='FAQ_Answers'), Object(uuid=_WeaviateUUIDInt('f2791143-c2ae-43d2-9917-1e57c0a25942'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_cons